## Model for Drone Steering

In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
#from category_encoders.one_hot import OneHotEncoder
#from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import Imputer
#from category_encoders.ordinal import OrdinalEncoder

from __future__ import print_function
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.constraints import maxnorm

import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import seaborn as sns

from pandas.api.types import is_numeric_dtype

import warnings
warnings.filterwarnings("ignore")
import ipytest.magics
import pytest
# set the file name (required)
__file__ = 'drone_pos_model.ipynb'

Using TensorFlow backend.


In [2]:
class Shuffler(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x): #x is df
        x=x.loc[np.random.permutation(x.index)]
        
        return x
############################################################################################
class XCentralizer(BaseEstimator, TransformerMixin):
    
    def __init__(self, x_columns):
        self.x_columns = x_columns
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x): #x is df
        shift=x[["rightShoulder_x","leftShoulder_x","leftHip_x","rightHip_x"]].sum(axis=1)/4
        for col in self.x_columns:
            x[col] = x[col] - shift
        return x
############################################################################################
    
class YCentralizer(BaseEstimator, TransformerMixin):
    
    def __init__(self, y_columns):
        self.y_columns = y_columns
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x): #x is df
        shift=x[["rightShoulder_y","leftShoulder_y","leftHip_y","rightHip_y"]].sum(axis=1)/4
        for col in list(set(self.y_columns)-set(["label"])):
            x[col] = x[col] - shift
        return x
############################################################################################

class YScaler(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x): #x is df
        shoulder_y = x[["rightShoulder_y","leftShoulder_y"]].sum(axis=1)/2
        hip_y = x[["leftHip_y","rightHip_y"]].sum(axis=1)/2
        y_dist = hip_y - shoulder_y
        
        for col in list(set(x.columns)-set(["label"])):
            x[col] /= y_dist
        return x

### Inspect Train Data

In [4]:
!pwd

/c/Users/p.schambach/Desktop/DSR/drone_steering/models


In [6]:
#df = pd.read_csv("video_001.csv", delimiter=',')
# Christian's video is less noisy. Therefore I only train the model with his data at the moment. 
# acc increased 5 % taking his video camparing to all videos.
#path = "all_videos_posture_steptime50_checksum8160"
#path = "video_Christian_posture_steptime50_checksum8160"
path = "video_all_posture_steptime50_checksum8160"
df = pd.read_csv("../data/" + path + ".csv",low_memory=False)
#df=df.drop([5557], axis=0)
#type(df.leftShoulder_x)
#df.info()
df.head(3)


,leftShoulder_x,leftShoulder_y,rightShoulder_x,rightShoulder_y,leftElbow_x,leftElbow_y,rightElbow_x,rightElbow_y,leftWrist_x,leftWrist_y,rightWrist_x,rightWrist_y,leftHip_x,leftHip_y,rightHip_x,rightHip_y,label
0,0.49250,0.18750,0.4000,0.18250,0.5050,0.26000,0.34375,0.19500,0.50875,0.33875,0.26625,0.16875,0.46500,0.34375,0.41125,0.34625,1
1,0.49250,0.18875,0.4025,0.18625,0.5075,0.25875,0.33250,0.19750,0.50000,0.33750,0.27625,0.17500,0.46750,0.33625,0.40875,0.33750,1
2,0.49125,0.19000,0.4025,0.17875,0.5050,0.26125,0.33500,0.19875,0.51125,0.33500,0.26375,0.16875,0.46375,0.33875,0.40875,0.33750,1


In [7]:
df1=df.dropna().drop_duplicates()
df1.shape

(3719, 17)

In [8]:
# Checking that we don't have any null values
assert df1.isnull().all().all() == False

In [10]:
x=df1.copy()
#x=x.drop(["label"], axis = 1)
x_cols = ['leftShoulder_x', 'rightShoulder_x',
        'leftElbow_x', 'rightElbow_x',
        'leftWrist_x', 'rightWrist_x',
        'leftHip_x', 'rightHip_x']
#xtrans = XCentralizer(x_cols)
#x = xtrans.transform(x)

y_cols = list(set(x.columns)-set(x_cols))
#print(y_cols)
#ytrans = YCentralizer(y_cols)
#x = ytrans.transform(x)

#ytrans = YScaler()
#x = ytrans.transform(x)
x[:2]

,leftShoulder_x,leftShoulder_y,rightShoulder_x,rightShoulder_y,leftElbow_x,leftElbow_y,rightElbow_x,rightElbow_y,leftWrist_x,leftWrist_y,rightWrist_x,rightWrist_y,leftHip_x,leftHip_y,rightHip_x,rightHip_y,label
0,0.4925,0.18750,0.4000,0.18250,0.5050,0.26000,0.34375,0.1950,0.50875,0.33875,0.26625,0.16875,0.4650,0.34375,0.41125,0.34625,1
1,0.4925,0.18875,0.4025,0.18625,0.5075,0.25875,0.33250,0.1975,0.50000,0.33750,0.27625,0.17500,0.4675,0.33625,0.40875,0.33750,1


In [11]:
from sklearn.pipeline import make_pipeline, make_union

processing_pipeline = make_pipeline(
    XCentralizer(x_cols),
    YCentralizer(y_cols), 
    YScaler(),
    Shuffler()
    )

In [12]:
processed_df = processing_pipeline.fit_transform(x)

In [13]:
%%run_pytest[clean]
def test_processingpipeline():
    # remember, this first pipeline only acts on the features, not the target.
    processed_df = processing_pipeline.fit_transform(x)
    
    # check for data leakage
    assert x.shape[0] == processed_df.shape[0]

================================================= test session starts =================================================
platform win32 -- Python 3.6.6, pytest-3.8.2, py-1.7.0, pluggy-0.7.1
rootdir: C:\Users\p.schambach\Desktop\DSR\drone_steering\models, inifile:
plugins: palladium-1.2.0
collected 1 item

drone_pos_model.py .                                                                                             [100%]

============================================== 1 passed in 0.14 seconds ===============================================


### Training Model

In [ ]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

# Model Architecture

In [34]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from math import sqrt
import matplotlib.pyplot as plt

In [15]:
from keras import models, layers
from keras.models import Model
from keras.layers import Input, Dense
from keras import optimizers, losses, metrics

def create_model():
    #default vaues
    #activation="relu"
    #optimizer="adam"
    lr=0.01
    #momentum=0
    #creat model
    model = models.Sequential()
    model.add(layers.Dense(
        32, 
        activation="relu", 
        input_shape=(16, )))
    model.add(layers.Dense(15, activation="relu"))
    model.add(layers.Dense(5, activation="softmax")) #is a fast rectifier
    #model.summary()   

    model.compile(
        optimizer=optimizers.RMSprop(lr=0.01),
        loss=losses.categorical_crossentropy,
        metrics=["accuracy"] 
    )
    
    return model

In [66]:
model_classifier = KerasClassifier(build_fn=create_model, epochs = 20, batch_size = 16)
type(model_classifier)


keras.wrappers.scikit_learn.KerasClassifier

In [46]:
x = df1.copy()

processed_df = processing_pipeline.transform(x)

x_train = processed_df.drop("label", axis=1)
y_train = processed_df["label"]



In [74]:
finalpipeline = (make_pipeline(processing_pipeline,model_classifier))
res = finalpipeline.fit(x_train, y_train)
type(res)

Epoch 1/20
3719/3719 [==============================] - 1s 361us/step - loss: 1.5779 - acc: 0.2649
Epoch 2/20
3719/3719 [==============================] - 0s 99us/step - loss: 1.5718 - acc: 0.2624: 0s - loss: 1.5594 - acc:
Epoch 3/20
3719/3719 [==============================] - 0s 99us/step - loss: 1.5687 - acc: 0.2670
Epoch 4/20
3719/3719 [==============================] - 0s 79us/step - loss: 1.5686 - acc: 0.2670
Epoch 5/20
3719/3719 [==============================] - 0s 74us/step - loss: 1.5670 - acc: 0.2748
Epoch 6/20
3719/3719 [==============================] - 0s 69us/step - loss: 1.5682 - acc: 0.2721
Epoch 7/20
3719/3719 [==============================] - 0s 66us/step - loss: 1.5680 - acc: 0.2646
Epoch 8/20
3719/3719 [==============================] - 0s 66us/step - loss: 1.5679 - acc: 0.2735
Epoch 9/20
3719/3719 [==============================] - 0s 66us/step - loss: 1.5666 - acc: 0.2713
Epoch 10/20
3719/3719 [==============================] - 0s 67us/step - loss: 1.5666 - acc:

sklearn.pipeline.Pipeline

In [80]:
#Use sckit_learn to grid search
activation = ['relu']
lr = [0.0001, 0.001, 0.1]
neurons = [1, 5, 10, 15, 20, 25, 30]
optimizer = ['RMSprop','Adam'] #['SGD', 'Adgrad', 'Adadelta', 'Adamax', 'Nadam']
######## grid serach epochs, batch_size
epochs = [10,20]
batch_size = [16,32]
param_grid = {"kerasclassifier__epochs": epochs, "kerasclassifier__batch_size": batch_size}



#######################################################
grid_search = GridSearchCV(estimator=finalpipeline, param_grid=param_grid, n_jobs=1)


In [76]:
print(finalpipeline.get_params().keys(),'\n')
print(model_classifier.get_params().keys(),'\n')
print(model_classifier.get_params()["epochs"])

dict_keys(['memory', 'steps', 'pipeline', 'kerasclassifier', 'pipeline__memory', 'pipeline__steps', 'pipeline__xcentralizer', 'pipeline__ycentralizer', 'pipeline__yscaler', 'pipeline__shuffler', 'pipeline__xcentralizer__x_columns', 'pipeline__ycentralizer__y_columns', 'kerasclassifier__epochs', 'kerasclassifier__batch_size', 'kerasclassifier__build_fn']) 

dict_keys(['epochs', 'batch_size', 'build_fn']) 

20


In [81]:
grid_result = grid_search.fit(x_train, y_train)

Epoch 1/10
2479/2479 [==============================] - 1s 411us/step - loss: 0.9607 - acc: 0.6136
Epoch 2/10
2479/2479 [==============================] - 0s 71us/step - loss: 0.5781 - acc: 0.7943
Epoch 3/10
2479/2479 [==============================] - 0s 67us/step - loss: 0.4952 - acc: 0.8249
Epoch 4/10
2479/2479 [==============================] - 0s 67us/step - loss: 0.4356 - acc: 0.8548
Epoch 5/10
2479/2479 [==============================] - 0s 67us/step - loss: 0.4035 - acc: 0.8520
Epoch 6/10
2479/2479 [==============================] - 0s 66us/step - loss: 0.3723 - acc: 0.8790
Epoch 7/10
2479/2479 [==============================] - 0s 69us/step - loss: 0.3554 - acc: 0.8802
Epoch 8/10
2479/2479 [==============================] - 0s 67us/step - loss: 0.3636 - acc: 0.8834
Epoch 9/10
2479/2479 [==============================] - 0s 67us/step - loss: 0.3394 - acc: 0.8883
Epoch 10/10
2479/2479 [==============================] - 0s 42us/step
Epoch 1/10
2479/2479 [=========================

2480/2480 [==============================] - 0s 82us/step - loss: 0.5390 - acc: 0.8250
Epoch 8/20
2480/2480 [==============================] - 0s 80us/step - loss: 0.5228 - acc: 0.8302
Epoch 9/20
2480/2480 [==============================] - 0s 81us/step - loss: 0.5125 - acc: 0.8387: 0s - loss: 0.5320 - acc: 0.82
Epoch 10/20
2480/2480 [==============================] - 0s 78us/step - loss: 0.4770 - acc: 0.8423
Epoch 11/20
2480/2480 [==============================] - 0s 81us/step - loss: 0.4778 - acc: 0.8411
Epoch 12/20
2480/2480 [==============================] - 0s 72us/step - loss: 0.4551 - acc: 0.8556
Epoch 13/20
2480/2480 [==============================] - 0s 68us/step - loss: 0.4628 - acc: 0.8444
Epoch 14/20
2480/2480 [==============================] - 0s 67us/step - loss: 0.4363 - acc: 0.8593
Epoch 15/20
2480/2480 [==============================] - 0s 67us/step - loss: 0.4359 - acc: 0.8617
Epoch 16/20
2480/2480 [==============================] - 0s 65us/step - loss: 0.4261 - acc: 

2479/2479 [==============================] - 0s 39us/step - loss: 0.3396 - acc: 0.8911
Epoch 14/20
2479/2479 [==============================] - 0s 39us/step - loss: 0.3273 - acc: 0.8955
Epoch 15/20
2479/2479 [==============================] - 0s 38us/step - loss: 0.3236 - acc: 0.8951
Epoch 16/20
2479/2479 [==============================] - 0s 39us/step - loss: 0.3131 - acc: 0.8971
Epoch 17/20
2479/2479 [==============================] - 0s 37us/step - loss: 0.3146 - acc: 0.8996
Epoch 18/20
2479/2479 [==============================] - 0s 36us/step - loss: 0.3118 - acc: 0.9036
Epoch 19/20
2479/2479 [==============================] - 0s 37us/step - loss: 0.3164 - acc: 0.8951
Epoch 20/20
2479/2479 [==============================] - 0s 25us/step
Epoch 1/20
2480/2480 [==============================] - 1s 455us/step - loss: 1.1937 - acc: 0.5347
Epoch 2/20
2480/2480 [==============================] - 0s 37us/step - loss: 0.8962 - acc: 0.6996
Epoch 3/20
2480/2480 [==============================

In [82]:
grid_result.cv_results_

{'mean_fit_time': array([ 8.05132333, 10.2471091 ,  7.60112564,  8.76963719]),
 'std_fit_time': array([0.41156132, 0.23214369, 0.22008464, 0.20465642]),
 'mean_score_time': array([5.17745701, 5.40219625, 5.70902673, 5.86339434]),
 'std_score_time': array([0.12477619, 0.06065127, 0.11324876, 0.03765009]),
 'param_kerasclassifier__batch_size': masked_array(data=[16, 16, 32, 32],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kerasclassifier__epochs': masked_array(data=[10, 20, 10, 20],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'kerasclassifier__batch_size': 16, 'kerasclassifier__epochs': 10},
  {'kerasclassifier__batch_size': 16, 'kerasclassifier__epochs': 20},
  {'kerasclassifier__batch_size': 32, 'kerasclassifier__epochs': 10},
  {'kerasclassifier__batch_size': 32, 'kerasclassifier__epochs': 20}],
 'split0_test_score': array([0.73790323, 0.68548387, 0.65

In [ ]:
##############################################################
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
history = model_classifier.fit(x_train, y_train, epochs=100, batch_size=30)

In [ ]:

plt.plot(history.history["loss"], label=["loss"]) #play with hyperparameters to see the changes
plt.legend()
plt.show()
plt.close()

plt.plot(history.history["acc"],  label=["acc"])
plt.legend()
plt.show()
plt.close()



In [ ]:
test_loss, test_acc = model_classifier.evaluate(x_val, y_val)
print("Loss / Accuracy Evaluation")
print("--------------------------")
print("Loss:     " + str(round(test_loss,5)))
print("Accuracy: " + str(round(test_acc,5)))

In [ ]:
val_y_pred = model_classifier.predict(x_val)

In [ ]:
val_y_pred[10]

In [ ]:
y_val[10]

### Save Model in tensorflow.js Format

The tensorflowjs library can't be installed directly with pip / conda due to conflicting dependencies. Best is to set up a new environment explicitly for this and install tensorflowjs with the following commands:

```
pip install tensorflow==1.11.0rc2 h5py numpy keras
pip install --no-deps tensorflowjs
```

In [ ]:
! pip install tensorflow==1.11.0rc2 h5py numpy keras
! pip install --no-deps tensorflowjs

In [ ]:
import tensorflowjs as tfjs

In [ ]:
tfjs.converters.save_keras_model(model, 'model_tfjs')

We need to adapt the two files as follows in order for them to work on Azure:
* add a file extension .pb to the file with no extension (otherwise Azure blocks it from viewing)
* adapt the automatically generated model.json to reflect the extension .pb